In [2]:
# open all files in game_states directory

import os
import pandas as pd
import duckdb


In [3]:
def do_query(query):
    with duckdb.connect('local2.db') as con:
        return con.execute(query).fetchdf()
def do_query_obj(query):
    with duckdb.connect('local2.db') as con:
        return con.execute(query).fetchall()

In [68]:
path = "/home/raph/Documents/GitHub/RoyalUr-Python"
file_names = []
for filename in os.listdir(path):
    if filename.endswith('.txt'):
        file_names.append(filename)

# sort the file in numerical order
file_names.sort(key=lambda x: int(x.split('_')[2].split('.')[0]))

do_query("DROP TABLE boardstate_lut;")
table_exists = False

for filename in file_names:
    print(filename)
    file_contents = ""
    with open(os.path.join(path, filename), 'r') as f:
        file_contents = f.read()
    # split the file contents into a list of lines
    file_contents = file_contents.split('\n')
    # split each lines at spaces
    file_contents = [line.split() for line in file_contents if line != ""]
    # create a dataframe from the list of lists
    df = pd.DataFrame(file_contents)
    if not table_exists:
        with duckdb.connect('local.db') as con:
            con.execute("CREATE TABLE boardstate_lut AS SELECT * FROM df")
        table_exists = True
    else:
        with duckdb.connect('local.db') as con:
            con.execute("INSERT INTO boardstate_lut SELECT * FROM df")

game_states_1.txt
game_states_2.txt
game_states_3.txt
game_states_4.txt
game_states_5.txt
game_states_6.txt
game_states_7.txt
game_states_8.txt
game_states_9.txt
game_states_10.txt
game_states_11.txt
game_states_12.txt
game_states_13.txt
game_states_14.txt
game_states_15.txt
game_states_16.txt
game_states_17.txt
game_states_18.txt
game_states_19.txt
game_states_20.txt
game_states_21.txt
game_states_22.txt
game_states_23.txt
game_states_24.txt
game_states_25.txt
game_states_26.txt
game_states_27.txt
game_states_28.txt
game_states_29.txt
game_states_30.txt
game_states_31.txt
game_states_32.txt
game_states_33.txt
game_states_34.txt
game_states_35.txt
game_states_36.txt
game_states_37.txt
game_states_38.txt
game_states_39.txt
game_states_40.txt
game_states_41.txt
game_states_42.txt
game_states_43.txt
game_states_44.txt
game_states_45.txt
game_states_46.txt
game_states_47.txt
game_states_48.txt
game_states_49.txt
game_states_50.txt
game_states_51.txt
game_states_52.txt
game_states_53.txt
ga

In [70]:
with duckdb.connect('local.db') as con:
    con.execute("ALTER TABLE boardstate_lut RENAME COLUMN '0' TO boardstate")
    con.execute("ALTER TABLE boardstate_lut RENAME COLUMN '1' TO light_score")
    con.execute("ALTER TABLE boardstate_lut RENAME COLUMN '2' TO dark_score")
    con.execute("ALTER TABLE boardstate_lut RENAME COLUMN '3' TO lut_index")

In [71]:
do_query("COPY boardstate_lut TO 'boardstate_lut.parquet' (FORMAT PARQUET);")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [72]:
do_query("DROP TABLE boardstate_lut_expanded;")

,Success


In [73]:
# boardstate column contains something like "..D..D..D.............L."
# we need to convert this to a series of columns, where each column is a position on the board.

"""
..D..D..D.............L.

becomes
ABC
..D 1 - 1-3
..D 2 - 4-6
..D 3 - 7-9
... 4 - 10-12
 .  5 - 14
 .  6 - 17
... 7 - 19-21
.L. 8 - 22-24

"""

# queries to convert the boardstate to a series of columns
do_query("""
CREATE TABLE boardstate_lut_expanded AS
SELECT
    lut_index,
    light_score,
    dark_score,
    boardstate,
    SUBSTRING(boardstate, 1, 1) AS A1,
    SUBSTRING(boardstate, 2, 1) AS B1,
    SUBSTRING(boardstate, 3, 1) AS C1,
    SUBSTRING(boardstate, 4, 1) AS A2,
    SUBSTRING(boardstate, 5, 1) AS B2,
    SUBSTRING(boardstate, 6, 1) AS C2,
    SUBSTRING(boardstate, 7, 1) AS A3,
    SUBSTRING(boardstate, 8, 1) AS B3,
    SUBSTRING(boardstate, 9, 1) AS C3,
    SUBSTRING(boardstate, 10, 1) AS A4,
    SUBSTRING(boardstate, 11, 1) AS B4,
    SUBSTRING(boardstate, 12, 1) AS C4,
    SUBSTRING(boardstate, 14, 1) AS B5,
    SUBSTRING(boardstate, 17, 1) AS B6,
    SUBSTRING(boardstate, 19, 1) AS A7,
    SUBSTRING(boardstate, 20, 1) AS B7,
    SUBSTRING(boardstate, 21, 1) AS C7,
    SUBSTRING(boardstate, 22, 1) AS A8,
    SUBSTRING(boardstate, 23, 1) AS B8,
    SUBSTRING(boardstate, 24, 1) AS C8
FROM boardstate_lut
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [74]:
do_query("DROP TABLE boardstate_lut_expanded_int;")

,Success


In [75]:
# convert columns A1 to C8 to integers where L = 1, D = 2, . = 0 in new table in batches

do_query("""
CREATE TABLE boardstate_lut_expanded_int AS
SELECT
    lut_index,
    light_score,
    dark_score,
    boardstate,
    CAST(CASE WHEN A1 = 'L' THEN 1 WHEN A1 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS A1,
    CAST(CASE WHEN B1 = 'L' THEN 1 WHEN B1 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B1,
    CAST(CASE WHEN C1 = 'L' THEN 1 WHEN C1 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS C1,
    CAST(CASE WHEN A2 = 'L' THEN 1 WHEN A2 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS A2,
    CAST(CASE WHEN B2 = 'L' THEN 1 WHEN B2 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B2,
    CAST(CASE WHEN C2 = 'L' THEN 1 WHEN C2 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS C2,
    CAST(CASE WHEN A3 = 'L' THEN 1 WHEN A3 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS A3,
    CAST(CASE WHEN B3 = 'L' THEN 1 WHEN B3 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B3,
    CAST(CASE WHEN C3 = 'L' THEN 1 WHEN C3 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS C3,
    CAST(CASE WHEN A4 = 'L' THEN 1 WHEN A4 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS A4,
    CAST(CASE WHEN B4 = 'L' THEN 1 WHEN B4 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B4,
    CAST(CASE WHEN C4 = 'L' THEN 1 WHEN C4 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS C4,
    CAST(CASE WHEN B5 = 'L' THEN 1 WHEN B5 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B5,
    CAST(CASE WHEN B6 = 'L' THEN 1 WHEN B6 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B6,
    CAST(CASE WHEN A7 = 'L' THEN 1 WHEN A7 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS A7,
    CAST(CASE WHEN B7 = 'L' THEN 1 WHEN B7 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B7,
    CAST(CASE WHEN C7 = 'L' THEN 1 WHEN C7 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS C7,
    CAST(CASE WHEN A8 = 'L' THEN 1 WHEN A8 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS A8,
    CAST(CASE WHEN B8 = 'L' THEN 1 WHEN B8 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS B8,
    CAST(CASE WHEN C8 = 'L' THEN 1 WHEN C8 = 'D' THEN 2 ELSE 0 END AS INTEGER) AS C8
FROM boardstate_lut_expanded
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [76]:
# show the first 10 rows of the new table
do_query("SELECT COUNT(*), lut_index FROM boardstate_lut_expanded_int GROUP BY lut_index LIMIT 10")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star(),lut_index
0,1,25390210
1,1,25390266
2,1,25390222
3,1,25390273
4,1,25390305
5,1,25390285
6,1,25390392
7,1,25390356
8,1,25390343
9,1,25390418


In [80]:
do_query("SELECT * FROM boardstate_lut_expanded_int WHERE lut_index = 8718052")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,C2,A3,B3,C3,A4,B4,C4,B5,B6,A7,B7,C7,A8,B8,C8
0,8718052,4,3,.L..D...D....D..L.L....D,0,1,0,0,2,0,0,0,2,0,0,0,2,1,1,0,0,0,0,2


In [79]:
# create unique index on the new table for column lut_index
do_query("CREATE UNIQUE INDEX lut_index_index ON boardstate_lut_expanded_int(lut_index)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count


In [81]:
# show all the columns from dataframe
pd.set_option('display.max_columns', None)
do_query("SELECT * FROM boardstate_lut_expanded LIMIT 10")

,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,C2,A3,B3,C3,A4,B4,C4,B5,B6,A7,B7,C7,A8,B8,C8
0,32,7,6,.......................D,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,D
1,16,7,6,....................D...,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,D,.,.,.
2,48,7,5,....................D..D,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,D,.,.,D
3,8,7,6,...........D............,.,.,.,.,.,.,.,.,.,.,.,D,.,.,.,.,.,.,.,.
4,40,7,5,...........D...........D,.,.,.,.,.,.,.,.,.,.,.,D,.,.,.,.,.,.,.,D
5,24,7,5,...........D........D...,.,.,.,.,.,.,.,.,.,.,.,D,.,.,.,.,D,.,.,.
6,56,7,4,...........D........D..D,.,.,.,.,.,.,.,.,.,.,.,D,.,.,.,.,D,.,.,D
7,4,7,6,........D...............,.,.,.,.,.,.,.,.,D,.,.,.,.,.,.,.,.,.,.,.
8,36,7,5,........D..............D,.,.,.,.,.,.,.,.,D,.,.,.,.,.,.,.,.,.,.,D
9,20,7,5,........D...........D...,.,.,.,.,.,.,.,.,D,.,.,.,.,.,.,.,D,.,.,.


In [9]:
# load boardstate_lut.parquet into boardstate_lut
do_query("CREATE TABLE boardstate_lut AS SELECT * FROM parquet_scan('boardstate_lut.parquet');")
#do_query("COPY boardstate_lut TO 'boardstate_lut.parquet' (FORMAT PARQUET);")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [10]:
do_query("DROP TABLE boardstate_lut_expanded")

,Success


In [35]:

from royalur import LutAgent
from huggingface_hub import hf_hub_download

REPO_ID = "sothatsit/RoyalUr"
FILENAME = "finkel.rgu"
filename = hf_hub_download(
    repo_id=REPO_ID, filename=FILENAME, cache_dir=os.getenv("HF_CACHE_DIR", None)
)
lut_player = LutAgent(filename)
lut = lut_player.lut


In [66]:
do_query("DROP TABLE IF EXISTS lut;")
do_query("CREATE TABLE lut(lut_index INTEGER, lut_value DOUBLE);")
numpy_keys = lut.keys_as_numpy(0)
numpy_values = lut.values_as_numpy(0)
lut_df = pd.DataFrame({'lut_index': numpy_keys, 'lut_value': numpy_values / 65535})
lut_df.lut_index = lut_df.lut_index.astype(int)
with duckdb.connect('local.db') as con:
    con.execute("INSERT INTO lut SELECT * FROM lut_df")

In [82]:
do_query("SELECT * FROM lut WHERE lut_value < 1 AND lut_value > 0 LIMIT 10")

,lut_index,lut_value
0,129,0.877470
1,130,0.886000
2,131,0.965682
3,132,0.894087
4,133,0.967208
5,134,0.967422
6,135,0.988190
7,136,0.907332
8,137,0.970886
9,138,0.971679


In [85]:
do_query("SELECT lut_value, boardstate_lut_expanded_int.* FROM lut INNER JOIN boardstate_lut_expanded_int ON lut.lut_index = boardstate_lut_expanded_int.lut_index LIMIT 10")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,lut_value,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,C2,A3,B3,C3,A4,B4,C4,B5,B6,A7,B7,C7,A8,B8,C8
0,0.641428,138172,2,1,....L..LD.LD.D..L..LD.DD,0,0,0,0,1,0,0,1,2,0,1,2,2,1,0,1,2,0,2,2
1,0.441398,138114,2,4,....LD.L..L..D..L..L..D.,0,0,0,0,1,2,0,1,0,0,1,0,2,1,0,1,0,0,2,0
2,0.460472,138146,2,3,....LD.L..L..D..L..L..DD,0,0,0,0,1,2,0,1,0,0,1,0,2,1,0,1,0,0,2,2
3,0.495247,138130,2,3,....LD.L..L..D..L..LD.D.,0,0,0,0,1,2,0,1,0,0,1,0,2,1,0,1,2,0,2,0
4,0.533303,138162,2,2,....LD.L..L..D..L..LD.DD,0,0,0,0,1,2,0,1,0,0,1,0,2,1,0,1,2,0,2,2
5,0.581079,138154,2,2,....LD.L..LD.D..L..L..DD,0,0,0,0,1,2,0,1,0,0,1,2,2,1,0,1,0,0,2,2
6,0.618708,138138,2,2,....LD.L..LD.D..L..LD.D.,0,0,0,0,1,2,0,1,0,0,1,2,2,1,0,1,2,0,2,0
7,0.641032,138170,2,1,....LD.L..LD.D..L..LD.DD,0,0,0,0,1,2,0,1,0,0,1,2,2,1,0,1,2,0,2,2
8,0.548913,138118,2,3,....LD.LD.L..D..L..L..D.,0,0,0,0,1,2,0,1,2,0,1,0,2,1,0,1,0,0,2,0
9,0.564874,138150,2,2,....LD.LD.L..D..L..L..DD,0,0,0,0,1,2,0,1,2,0,1,0,2,1,0,1,0,0,2,2


In [93]:
offset = 0
limit = 1000000

with duckdb.connect('local.db') as con:
    #con.execute("CREATE UNIQUE INDEX lut_index_index ON boardstate_lut_expanded_int(lut_index)")
    con.execute("CREATE UNIQUE INDEX lut_lut_index_index ON lut(lut_index)")
    con.execute("""
CREATE TABLE boardstate_lut_expanded_int_values AS
SELECT lut.lut_value, boardstate_lut_expanded_int.*
FROM lut
INNER JOIN boardstate_lut_expanded_int
ON lut.lut_index = boardstate_lut_expanded_int.lut_index
""")
    #while True:
    #    r = con.execute(f"SELECT * FROM boardstate_lut_expanded_int LIMIT {limit} OFFSET {offset}")
    #    if r.shape[0] == 0:
    #        break
    #    lut_values = []
    #    for i in range(r.shape[0]):
    #        lut_values.append(lut.lookup(0, int(r.lut_index[i])))
    #    r['lut_value'] = lut_values
    #    with duckdb.connect('local.db') as con:
    #        con.execute("INSERT INTO boardstate_lut_expanded_int_values SELECT * FROM r")
    #    offset += limit
    #    print(offset)
        



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [187]:
do_query("SELECT * FROM boardstate_lut_expanded_int_values WHERE light_piece_left_to_play > 6 LIMIT 10")

,lut_value,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,C2,A3,B3,C3,A4,B4,C4,B5,B6,A7,B7,C7,A8,B8,C8,light_piece_count,dark_piece_count,light_piece_left_to_play,dark_piece_left_to_play
0,0.000031,1879048224,0,6,.......................D,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,7,0
1,0.000046,1879048208,0,6,....................D...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,7,0
2,0.000061,1879048240,0,5,....................D..D,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,2,7,0
3,0.063767,1879048200,0,6,...........D............,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,7,0
4,0.064668,1879048232,0,5,...........D...........D,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,2,7,0
5,0.071473,1879048216,0,5,...........D........D...,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,2,7,0
6,0.054948,1879048196,0,6,........D...............,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,7,0
7,0.054932,1879048228,0,5,........D..............D,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,2,7,0
8,0.062516,1879048212,0,5,........D...........D...,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,2,7,0
9,0.070008,1879048244,0,4,........D...........D..D,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,2,0,3,7,0


In [156]:
# alter table to convert light_score and dark_score to integers
do_query("ALTER TABLE boardstate_lut_expanded_int_values ALTER COLUMN light_score TYPE INTEGER;")
do_query("ALTER TABLE boardstate_lut_expanded_int_values ALTER COLUMN dark_score TYPE INTEGER;")

,Success


In [43]:
# drop unique index on lut_index
do_query("DROP INDEX boardstate_lut_expanded_int_values_lut_index_index;")
do_query("ALTER TABLE boardstate_lut_expanded_int_values ALTER COLUMN lut_index TYPE INTEGER;")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [178]:
# add a column that counts the number of pieces on the board. 0 is empty, 1 is light, 2 is dark
do_query("ALTER TABLE boardstate_lut_expanded_int_values ADD COLUMN light_piece_count INTEGER;")
do_query("ALTER TABLE boardstate_lut_expanded_int_values ADD COLUMN dark_piece_count INTEGER;")


,Success


In [179]:

do_query("""
UPDATE boardstate_lut_expanded_int_values
SET light_piece_count = CASE WHEN A1 = 1 THEN 1 ELSE 0 END + CASE WHEN B1 = 1 THEN 1 ELSE 0 END + CASE WHEN C1 = 1 THEN 1 ELSE 0 END + CASE WHEN A2 = 1 THEN 1 ELSE 0 END + CASE WHEN B2 = 1 THEN 1 ELSE 0 END + CASE WHEN C2 = 1 THEN 1 ELSE 0 END + CASE WHEN A3 = 1 THEN 1 ELSE 0 END + CASE WHEN B3 = 1 THEN 1 ELSE 0 END + CASE WHEN C3 = 1 THEN 1 ELSE 0 END + CASE WHEN A4 = 1 THEN 1 ELSE 0 END + CASE WHEN B4 = 1 THEN 1 ELSE 0 END + CASE WHEN C4 = 1 THEN 1 ELSE 0 END + CASE WHEN B5 = 1 THEN 1 ELSE 0 END + CASE WHEN B6 = 1 THEN 1 ELSE 0 END + CASE WHEN A7 = 1 THEN 1 ELSE 0 END + CASE WHEN B7 = 1 THEN 1 ELSE 0 END + CASE WHEN C7 = 1 THEN 1 ELSE 0 END + CASE WHEN A8 = 1 THEN 1 ELSE 0 END + CASE WHEN B8 = 1 THEN 1 ELSE 0 END + CASE WHEN C8 = 1 THEN 1 ELSE 0 END;
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [182]:

do_query("""
UPDATE boardstate_lut_expanded_int_values
SET dark_piece_count = CASE WHEN A1 = 2 THEN 1 ELSE 0 END + CASE WHEN B1 = 2 THEN 1 ELSE 0 END + CASE WHEN C1 = 2 THEN 1 ELSE 0 END + CASE WHEN A2 = 2 THEN 1 ELSE 0 END + CASE WHEN B2 = 2 THEN 1 ELSE 0 END + CASE WHEN C2 = 2 THEN 1 ELSE 0 END + CASE WHEN A3 = 2 THEN 1 ELSE 0 END + CASE WHEN B3 = 2 THEN 1 ELSE 0 END + CASE WHEN C3 = 2 THEN 1 ELSE 0 END + CASE WHEN A4 = 2 THEN 1 ELSE 0 END + CASE WHEN B4 = 2 THEN 1 ELSE 0 END + CASE WHEN C4 = 2 THEN 1 ELSE 0 END + CASE WHEN B5 = 2 THEN 1 ELSE 0 END + CASE WHEN B6 = 2 THEN 1 ELSE 0 END + CASE WHEN A7 = 2 THEN 1 ELSE 0 END + CASE WHEN B7 = 2 THEN 1 ELSE 0 END + CASE WHEN C7 = 2 THEN 1 ELSE 0 END + CASE WHEN A8 = 2 THEN 1 ELSE 0 END + CASE WHEN B8 = 2 THEN 1 ELSE 0 END + CASE WHEN C8 = 2 THEN 1 ELSE 0 END;
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [184]:
# add a column that counts the number of pieces on the board. 0 is empty, 1 is light, 2 is dark
do_query("ALTER TABLE boardstate_lut_expanded_int_values ADD COLUMN light_piece_left_to_play INTEGER;")
do_query("ALTER TABLE boardstate_lut_expanded_int_values ADD COLUMN dark_piece_left_to_play INTEGER;")

do_query("""
UPDATE boardstate_lut_expanded_int_values
SET light_piece_left_to_play = 7 - light_piece_count - light_score;
""")

do_query("""
UPDATE boardstate_lut_expanded_int_values
SET dark_piece_left_to_play = 7 - dark_piece_count - dark_score;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [95]:
# save the table to parquet
do_query("COPY boardstate_lut_expanded_int_values TO 'boardstate_lut_expanded_int_values.parquet' (FORMAT PARQUET);")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [107]:
# get distinct values from all A1 to C8 columns

cols = ['A1', 'B1', 'C1', 'A2', 'B2', 'C2', 'A3', 'B3', 'C3', 'A4', 'B4', 'C4', 'B5', 'B6', 'A7', 'B7', 'C7', 'A8', 'B8', 'C8']

s = []
for col in cols:
    r = do_query(f"SELECT DISTINCT {col} FROM boardstate_lut_expanded_int_values")
    print(r)


   A1
0   0
1   1
   B1
0   0
1   2
2   1
   C1
0   0
1   2
   A2
0   0
1   1
   B2
0   0
1   2
2   1
   C2
0   0
1   2
   A3
0   0
1   1
   B3
0   2
1   0
2   1
   C3
0   2
1   0
   A4
0   0
1   1
   B4
0   2
1   1
2   0
   C4
0   0
1   2
   B5
0   1
1   0
2   2
   B6
0   0
1   1
2   2
   A7
0   1
1   0
   B7
0   0
1   2
2   1
   C7
0   0
1   2
   A8
0   0
1   1
   B8
0   0
1   1
2   2
   C8
0   2
1   0


In [53]:
to_convert = do_query_obj("""
SELECT boardstate, light_score, dark_score
FROM boardstate_lut_expanded_int_values 
WHERE light_score < 7 AND dark_score < 7 
LIMIT 1000
""")

In [9]:
# create unique constraint on boardstate_lut_expanded_int_values for column lut_index
do_query("CREATE UNIQUE INDEX boardstate_lut_expanded_int_values_lut_index_index ON boardstate_lut_expanded_int_values(lut_index)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count


In [29]:
from royalur.model.player import PlayerType
from royalur.lut.board_encoder import SimpleGameStateEncoding
encoding = SimpleGameStateEncoding()

def lut_get_encoded(current_state):
    current_state._turn = PlayerType.LIGHT
    return encoding.encode_game_state(current_state)


In [21]:
do_query("DROP TABLE lut_transition;")
do_query("CREATE TABLE lut_transition(current_state INTEGER, next_state INTEGER, dice INTEGER, next_player INTEGER);")

,Count


In [30]:
from royalur import Board
from royalur.model import StandardBoardShape, Piece, PlayerState
from royalur.model.player import PlayerType
from royalur.rules.state import WaitingForRollGameState, WinGameState
from royalur.game import Game
import cProfile


f = Game.create_finkel(pawns=7)
b = Board(StandardBoardShape())
char_to_player = {
    "L": PlayerType.LIGHT,
    "D": PlayerType.DARK,
}
"""
.DD 0-2
..D 3-5
.LD 6-8
..D 9-11
 L  12-14
 .  15-17
LD. 18-20
.D. 21-23
"""
light_path = [
    (1, 4),
    (1, 3),
    (1, 2),
    (1, 1),
    (2, 1),
    (2, 2),
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (1, 8),
    (1, 7),
]
light_path = [9, 6, 3, 0,  1, 4, 7, 10, 13, 16, 19, 22,  21, 18]
dark_path = [
    (3, 4),
    (3, 3),
    (3, 2),
    (3, 1),
    (2, 1),
    (2, 2),
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (3, 8),
    (3, 7),
]

dark_path = [11, 8, 5, 2,  1, 4, 7, 10, 13, 16, 19, 22,  23, 20]

char_to_path = {
    "L": light_path,
    "D": dark_path,
}
limit = 10000
offset = 0

profile = cProfile.Profile()
profile.enable()
while True:
    insert_buffer = []
    to_convert = do_query_obj(f"""
    SELECT boardstate, light_score, dark_score
    FROM boardstate_lut_expanded_int_values 
    WHERE light_score < 7 AND dark_score < 7 
    LIMIT {limit} OFFSET {offset}
    """)
    if len(to_convert) == 0:
        break
    offset += limit
    print(offset)
    with duckdb.connect('local2.db') as con:
        for board_state, light_score, dark_score in to_convert:
            #print(board_state, light_score, dark_score)
            pawns_in_play = {
                "L": 0,
                "D": 0,
            }
            for i, state in enumerate(board_state):
                if state == ".":
                    b._pieces[i] = None
                else:
                    path = char_to_path[state]
                    # find the index of the path
                    index = path.index(i)
                    b._pieces[i] = Piece(char_to_player[state], index)
                    pawns_in_play[state] += 1
            #print(pawns_in_play)
            light_pieces = 7 - pawns_in_play["L"] - light_score
            dark_pieces = 7 - pawns_in_play["D"] - dark_score
            #print(light_pieces, dark_pieces)

            light_player_state = PlayerState(PlayerType.LIGHT, light_pieces, light_score)
            dark_player_state = PlayerState(PlayerType.DARK, dark_pieces, dark_score)

            gs = WaitingForRollGameState(
                b,
                light_player_state,
                dark_player_state,
                PlayerType.LIGHT,
            )
            original_lut_state = lut_get_encoded(gs)
            insert_buffer.append((original_lut_state, original_lut_state, 0, 2))
            #print(gs.describe())
            for i in range(1, 5):
                j = f.copy()
                j.add_state(gs)
                #print(j.get_board())
                #print(j._states[-1].describe())
                #print(i)
                j.roll_dice(i)
                #print(j._states[-1])
                #print(j._states[-1].describe())
                state = j._states[-1]
                if type(state) == WaitingForRollGameState:
                    insert_buffer.append((original_lut_state, original_lut_state, i, 2))
                    continue
                moves = j.find_available_moves()
                #print("AFTER MOVES")
                for move in moves:
                    k = j.copy()
                    k.make_move(move)
                    state = k._states[-1]
                    #print(k.get_board())
                    #print(k._states[-1].describe())
                    #print(state._turn)
                    if type(state) == WinGameState:
                        next_player_int = -(1 if state._winner == PlayerType.LIGHT else 2)
                        new_lut_state = "null"
                    else:
                        next_player_int = 1 if state._turn == PlayerType.LIGHT else 2
                        new_lut_state = lut_get_encoded(state)
                    insert_buffer.append((original_lut_state, new_lut_state, i, next_player_int))
                    #new_board_state = con.execute(f"SELECT boardstate FROM boardstate_lut_expanded_int_values WHERE lut_index = {new_lut_state}").fetchall()[0][0]
                #print()
            #print()
        # generate INSERT INTO statement for the buffer
        insert_string = []
        for i, (original_lut_state, new_lut_state, dice, player) in enumerate(insert_buffer):
            insert_string.append(f"({original_lut_state}, {new_lut_state}, {dice}, {player})")
        insert_string = "INSERT INTO lut_transition(current_state, next_state, dice, next_player) VALUES " + ", ".join(insert_string)
        con.execute(insert_string)
        print("Done")
        break

profile.disable()
profile.print_stats(2)

10000
Done
         19176435 function calls in 5.586 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    6.752    3.376 interactiveshell.py:3541(run_code)
        2    0.000    0.000    6.752    3.376 {built-in method builtins.exec}
    40000    0.045    0.000    1.788    0.000 game.py:226(roll_dice)
    79270    0.033    0.000    1.696    0.000 game.py:255(make_move)
    79270    0.055    0.000    1.657    0.000 game.py:248(_make_move)
    40000    0.116    0.000    1.629    0.000 simple.py:227(apply_roll)
    79270    0.394    0.000    1.483    0.000 simple.py:269(apply_move)
    40000    0.386    0.000    1.356    0.000 simple.py:147(find_available_moves)
    89185    0.040    0.000    1.184    0.000 2211854293.py:5(lut_get_encoded)
    89185    0.118    0.000    1.145    0.000 board_encoder.py:88(encode_game_state)
    89185    0.070    0.000    0.990    0.000 board_encoder.py:82(encode_boar

In [89]:
do_query("SELECT * FROM lut_transition WHERE next_player = 2 LIMIT 10")

,current_state,next_state,dice,next_player
0,8563407,174799,1,2
1,8563407,8561103,1,2
2,8563407,42114575,2,2
3,8563407,8533455,3,2
4,8563407,42114447,3,2
5,8563407,42086927,4,2
6,8563407,25337167,4,2
7,8563456,174848,1,2
8,8563456,8561152,1,2
9,8563456,25340544,1,2


In [5]:
do_query("FORCE CHECKPOINT;")

,Success


In [6]:
# copy contents of boardstate_lut_expanded_int_values to a new database (local2.db)
do_query("COPY boardstate_lut_expanded_int_values TO 'boardstate_lut_expanded_int_values.parquet' (FORMAT PARQUET);")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:

with duckdb.connect('local2.db') as con:
    con.execute("CREATE TABLE boardstate_lut_expanded_int_values AS SELECT * FROM parquet_scan('boardstate_lut_expanded_int_values.parquet');")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [90]:
0.7/1000*138000000/60/60

26.833333333333332